# Requirements
* [Hugging Face Transformers](https://huggingface.co/docs/transformers/en/installation) - `pip install transformers`
* [PyTorch](https://pytorch.org/get-started/locally/) - `pip install torch torchvision torchaudio`

In [44]:
import os # Allows us to retrieve file paths
import pandas # Needed to open and read the dataset

def get_path_to_file(file: str) -> str:
    ''' Returns the path to a file in a working directory.
        Note that function will still return a path even if the file
        doesn't exist in that directory.'''
    return os.path.realpath(file)

re0_data = pandas.read_csv(get_path_to_file('RE0_Data.csv'), encoding = 'utf-8')
re0_data

,Character,Gender,Line
0,Narrator,NaN,A small mid-western town in America: Raccoon C...
1,Man,Male,Really?
2,Woman,Female,"Hmm, do you think so too?"
3,Bald Man,Male,Yeah.
4,White Man,Male,.do about it?
...,...,...,...
228,Billy,Male,"Rebecca, hurry!"
229,Rebecca,Female,Hey that must be the old mansion Enrico was ta...
230,Rebecca,Female,"I guess it's time to say goodbye. Officially, ..."
231,Billy,Male,"Yeah, I'm just a zombie now."


In [45]:
from transformers import pipeline, Pipeline # Needed to run the model

def get_emotion(line: str, classifier: Pipeline, threshold: float) -> str:
    ''' Attempts to identify the primary emotion in a piece of text based
        on results returned by the model. 
        If the confidence score is below the threshold, return NaN instead. '''
    model_output = classifier(line)[0] # This returns a list of dictionaries -- we only want the first entry
    if (model_output['score'] < threshold):
        return pandas.NA
    return model_output['label']

## Results: robert-base-go_emotions
Link: https://huggingface.co/SamLowe/roberta-base-go_emotions

In [46]:
# https://huggingface.co/SamLowe/roberta-base-go_emotions 
classifier = pipeline(task = "text-classification", model="SamLowe/roberta-base-go_emotions")
re0_data = pandas.read_csv(get_path_to_file('RE0_Data.csv'), encoding = 'utf-8') # Reset
re0_data['Emotion'] = re0_data['Line'].apply(lambda x: get_emotion(x, classifier, 0.70))
re0_data = re0_data.dropna(subset = ['Emotion'])
re0_data

,Character,Gender,Line,Emotion
4,White Man,Male,.do about it?,neutral
5,Black Man,NaN,We should hold off for a while.,neutral
6,White Man,Male,I wonder how those RNP people will react to this?,curiosity
8,Man With Glasses,Male,(gasps),neutral
9,Man With Glasses,Male,(screams and leaps from his seat.),neutral
...,...,...,...,...
227,Rebecca,Female,Billy!,neutral
228,Billy,Male,"Rebecca, hurry!",neutral
229,Rebecca,Female,Hey that must be the old mansion Enrico was ta...,neutral
230,Rebecca,Female,"I guess it's time to say goodbye. Officially, ...",sadness


In [47]:
emotion_counts = re0_data.groupby('Character')['Emotion'].value_counts().reset_index(name = 'Count')
emotion_counts

,Character,Emotion,Count
0,Billy,neutral,25
1,Billy,caring,3
2,Billy,joy,2
3,Billy,excitement,1
4,Billy,fear,1
5,Billy,gratitude,1
6,Birkin,amusement,1
7,Birkin,confusion,1
8,Birkin,neutral,1
9,Black Man,neutral,1


## Results: BERT-Emotions-Classifier
Link: https://huggingface.co/ayoubkirouane/BERT-Emotions-Classifier

In [49]:
# https://huggingface.co/SamLowe/roberta-base-go_emotions 
classifier = pipeline(task = "text-classification", model="ayoubkirouane/BERT-Emotions-Classifier")
re0_data = pandas.read_csv(get_path_to_file('RE0_Data.csv'), encoding = 'utf-8') # Reset
re0_data['Emotion'] = re0_data['Line'].apply(lambda x: get_emotion(x, classifier, 0.70))
re0_data = re0_data.dropna(subset = ['Emotion'])
re0_data

,Character,Gender,Line,Emotion
0,Narrator,NaN,A small mid-western town in America: Raccoon C...,fear
2,Woman,Female,"Hmm, do you think so too?",anticipation
10,Rebecca,Female,It began as a simple investigation of some biz...,fear
11,Enrico,Male,What's going on?,disgust
13,Edward,Male,Ugh!!,anger
...,...,...,...,...
226,Billy,Male,"Hey, Queenie! Feast on this!",joy
227,Rebecca,Female,Billy!,joy
230,Rebecca,Female,"I guess it's time to say goodbye. Officially, ...",sadness
231,Billy,Male,"Yeah, I'm just a zombie now.",sadness


In [50]:
emotion_counts = re0_data.groupby('Character')['Emotion'].value_counts().reset_index(name = 'Count')
emotion_counts

,Character,Emotion,Count
0,Billy,anger,13
1,Billy,joy,7
2,Billy,optimism,6
3,Billy,fear,5
4,Billy,anticipation,3
5,Billy,sadness,2
6,Billy,disgust,1
7,Birkin,anger,2
8,Birkin,disgust,2
9,Birkin,joy,1
